# Currently missing things (will need testing)
- New packages (those would have a `File.status` of `created` iirc)
- Renamed packages (those would be [I think] like the above but with a status of `renamed` on the link)
- Deleted packages (I think that would be cleanly all templates that go `deleted`)

In [59]:
import github
from collections import defaultdict
import pathlib

In [60]:
with pathlib.Path("./tokenfile").open() as tokenfile:
    token = tokenfile.readline().strip()

In [69]:
gh = github.Github(login_or_token=token)
repo = gh.get_repo("void-linux/void-packages")

# the PR number goes here
pr = repo.get_pull(36432)

files = list(pr.get_files())

In [70]:
def ischange(line: str):
    return line.startswith(("+", "-"))
def patch_changes(patch: str):
    return list(filter(ischange, patch.splitlines()))

In [71]:
_labels = []
_pkgs: dict[str, list] = defaultdict(lambda: list())
for file in files:
    fn = file.filename
    if fn.startswith(("xbps-src", "common/", "etc/")):
        if fn == "common/shlibs":
            _labels += ["shlibs"]
            continue
        _labels += ["xbps-src"]
        continue
    if fn in ("Manual.md", "README.md", "CONTRIBUTING.md"):
        _labels += ["documentation"]
        continue
    if fn.startswith(("srcpkgs/")):
        fn = fn.removeprefix("srcpkgs/")
        fp = fn.split("/")
        pkg = fp[0]
        if len(fp) >= 2:
            # the file is in the package's directory
            if fp[1] == "template":
                match file.status:
                    case "added":
                        _pkgs[pkg] += ["new-package"]
                    case "removed":
                        _pkgs[pkg] += ["deletion"]
                    case _:
                        changes = patch_changes(file.patch)
                        for chg in changes:
                            if chg.startswith(("-version=", "+version=")):
                                _pkgs[pkg] += ["update"]
                                continue
                            if chg.startswith(("-revision=", "+revision=")):
                                _pkgs[pkg] += ["revbump"]
                            else:
                                _pkgs[pkg] += ["cleanup-fixups"]
            else:
                # the modified file is not the template
                _pkgs[pkg] += ["cleanup-fixups"]
        else:
            # we have a symlink to a package
            match file.status:
                case "added":
                    # Could be a pakcage being renamed or a new subpackage
                    _pkgs[pkg] += ["_link_new"]
                    target = patch_changes(file.patch)[0].removeprefix("+")
                    _pkgs[target] += ["_target_new"]
                case "removed":
                    _pkgs[pkg] += ["deletion"]

# Give preference 'update', since an update strongly implies
# a rev reset and cleanups/fixups for the new version
for p, l in _pkgs.items():
    print(p, l)  # TODO: REMOVE
    if "deletion" in l and "_link_new" in l:
        # If the package is a new link and TODO
        _labels += ["renamed"]
        continue
    if "new-package" in l and "_target_new" in l:
        # If the package both has a new template or is the target
        # of a new symlink, it is the result of a rename operation
        _labels += ["renamed"]
        continue
    if "deletion" in l:
        # The package is GONE, TURNED TO ETERNAL DUST
        _labels += ["deletion"]
        continue
    if "new-package" in l:
        # If the package is new, any other info is noise
        _labels += ["new-package"]
        continue
    if "update" in l:
        # In the case of package update, we don't care if that package
        # got minor change tab like revbump or cleanup-fixups
        _labels += ["update"]
        continue
    # We only have revbump and/or cleanup-fixups
    _labels += l
_labels

emacs-gtk2 ['deletion']
emacs ['cleanup-fixups', 'cleanup-fixups', 'cleanup-fixups', 'cleanup-fixups', 'cleanup-fixups', 'cleanup-fixups', 'cleanup-fixups', 'cleanup-fixups', 'cleanup-fixups', 'cleanup-fixups', 'cleanup-fixups', 'cleanup-fixups', 'cleanup-fixups', 'cleanup-fixups', 'cleanup-fixups', 'cleanup-fixups', 'cleanup-fixups', 'cleanup-fixups', 'cleanup-fixups', 'cleanup-fixups']
removed-packages ['cleanup-fixups']


['deletion',
 'cleanup-fixups',
 'cleanup-fixups',
 'cleanup-fixups',
 'cleanup-fixups',
 'cleanup-fixups',
 'cleanup-fixups',
 'cleanup-fixups',
 'cleanup-fixups',
 'cleanup-fixups',
 'cleanup-fixups',
 'cleanup-fixups',
 'cleanup-fixups',
 'cleanup-fixups',
 'cleanup-fixups',
 'cleanup-fixups',
 'cleanup-fixups',
 'cleanup-fixups',
 'cleanup-fixups',
 'cleanup-fixups',
 'cleanup-fixups',
 'cleanup-fixups']

### The labels to apply to the PR
The set deduplicates the lebels for us

In [72]:
set(_labels)

{'cleanup-fixups', 'deletion'}